# Otimização da Camada de Saída ($w_3, w_4, b_3$)

Nesta etapa, utilizaremos o **Gradiente Descendente** para ajustar **toda a camada de saída** da nossa rede neural. Nosso objetivo é encontrar os valores ideais para os pesos $w_3, w_4$ e para o viés $b_3$ simultaneamente.

A estrutura matemática da nossa rede (Forward Pass) é definida por:

$$
h_1 = \ln(1 + e^{(x \cdot w_1 + b_1)})
$$

$$
h_2 = \ln(1 + e^{(x \cdot w_2 + b_2)})
$$

$$
\hat{y} = (h_1 \cdot \mathbf{w_3} + h_2 \cdot \mathbf{w_4}) + \mathbf{b_3}
$$

**Onde:**
* **$x$**: Dado de entrada.
* **$h_1, h_2$**: Neurônios da camada oculta (ativados por Softplus).
* **$w_1, w_2, b_1, b_2$**: Parâmetros da camada oculta (que manteremos **congelados/fixos**).
* **$\mathbf{w_3}, \mathbf{w_4}$**: **Pesos de Saída** (vamos otimizar a influência de cada neurônio oculto na soma final).
* **$\mathbf{b_3}$**: **Viés Final** (vamos otimizar o deslocamento vertical).
* **$\hat{y}$**: A previsão final da rede.

O algoritmo agora calculará **três gradientes parciais** a cada iteração (um para cada parâmetro da saída), ajustando a inclinação e a altura da curva final para minimizar a diferença entre a previsão ($\hat{y}$) e os dados reais.

##Bibliotecas que usaremos

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

## Definição das variaveis com os dados que usaremos:

In [10]:
w1, w2 = 3.34, -3.53,
b1, b2 = -1.43, 0.57

b3 = 0
w3, w4 = 0.36 , 0.63

values_of_b3, values_of_w3, values_of_w4 = [], [], []
list_current_gradient_b3 = []
list_current_gradient_w3 = []
list_current_gradient_w4 = []
list_sum_of_squared_residuals = []

x_data = np.array([0, 0.5, 1])
y_data = np.array([0, 1, 0])
max_iterations = 1000
precision = 0.0001
learning_rate = 0.05

# Definição das funções
Para facilitar a leitura, vamos associar as variáveis do código aos símbolos matemáticos:

* **$y_i$**: `y_data` (Valor real).
* **$\hat{y}_i$**: `predictions` (Previsão da rede).
* **$h_1$**: `y_curve1` (Saída do neurônio oculto 1).
* **$h_2$**: `y_curve2` (Saída do neurônio oculto 2).
* **$w_3, w_4$**: `w3`, `w4` (Pesos que estamos treinando).
* **$b_3$**: `b3` (Viés que estamos treinando).



## 1. Equação da Rede Neural (Forward Pass)
Correspondente à função `neural_network`.

Esta função define a estrutura matemática da rede:
* **Calcula** a ativação dos neurônios ocultos usando a função **Softplus** ($\ln(1+e^z)$).
* **Combina** os resultados linearmente para gerar a saída.

$$
h_1 = \ln(1 + e^{(x \cdot w_1 + b_1)})
$$

$$
h_2 = \ln(1 + e^{(x \cdot w_2 + b_2)})
$$

**A saída final é:**

$$
\hat{y} = (h_1 \cdot w_3 + h_2 \cdot w_4) + b_3
$$

In [3]:
def neural_network(input_val, b3, w3, w4):
  y_curve1 = np.log(1 + np.exp(input_val*w1+b1))
  y_curve2 = np.log(1 + np.exp(input_val*w2+b2))
  y_curve = (y_curve1*w3+y_curve2*w4) +b3
  return y_curve

## 1. Gradiente do Viés ($b_3$)
Correspondente à função `calculate_gradient_b3`.

$$
\frac{\partial SSR}{\partial b_3} = \sum_{i=1}^{n} -2 (y_i - \hat{y}_i)
$$


In [4]:
def calculate_gradient_b3(x_data, b3, w3, w4, y_data):
  _, residuals, _, _ = _calculate_predictions_and_residuals(x_data, b3, w3, w4, y_data)
  return np.sum(-2 * residuals)

## 2. Gradiente do Peso $w_3$
Correspondente à função `calculate_gradient_w3`.
Pela Regra da Cadeia, o gradiente deste peso depende do erro multiplicado pela ativação do neurônio ao qual ele está conectado ($h_1$).

$$
\frac{\partial SSR}{\partial w_3} = \sum_{i=1}^{n} -2 (y_i - \hat{y}_i) \cdot h_1
$$


In [5]:
def calculate_gradient_w3(x_data, b3, w3, w4, y_data):
  _, residuals, y_curve1, _ = _calculate_predictions_and_residuals(x_data, b3, w3, w4, y_data)
  return np.sum(-2 * residuals * y_curve1)


## 3. Gradiente do Peso $w_4$
Correspondente à função `calculate_gradient_w4`.
De forma similar, este gradiente é ponderado pela ativação do segundo neurônio oculto ($h_2$).

$$
\frac{\partial SSR}{\partial w_4} = \sum_{i=1}^{n} -2 (y_i - \hat{y}_i) \cdot h_2
$$

In [6]:
def calculate_gradient_w4(x_data, b3, w3, w4, y_data):
  _, residuals, _, y_curve2 = _calculate_predictions_and_residuals(x_data, b3, w3, w4, y_data)
  return np.sum(-2 * residuals * y_curve2)

### Função Auxiliar: Forward Pass e Cálculo de Resíduos

Esta função centraliza a execução da rede neural e o cálculo do erro. Ela processa a entrada através das camadas ocultas, aplica os parâmetros da camada de saída ($w_3, w_4, b_3$) e compara o resultado com os dados reais.


A função retorna quatro valores essenciais para a etapa de retropropagação (backpropagation):

1.  **`predictions`** ($\hat{y}$): A estimativa final da rede.
2.  **`residuals`** ($y - \hat{y}$): A diferença entre o valor real e o previsto. Este vetor é a base para o cálculo do gradiente de todos os parâmetros.
3.  **`y_curve1` e `y_curve2`** ($h_1, h_2$): As ativações dos neurônios da camada oculta.

In [7]:
def _calculate_predictions_and_residuals(x_data, b3, w3, w4, y_data):
  y_curve1 = np.log(1 + np.exp(x_data*w1+b1))
  y_curve2 = np.log(1 + np.exp(x_data*w2+b2))
  predictions = (y_curve1*w3+y_curve2*w4) +b3
  residuals = y_data - predictions
  return predictions, residuals, y_curve1, y_curve2

## Loop de Treinamento: Otimizando $w_3, w_4$ e $b_3$

A rede neural aprende os melhores valores para os parâmetros da camada de saída ($w_3, w_4$ e $b_3$) repetindo o seguinte ciclo:

1.  **Forward Pass:** A rede faz uma previsão usando os valores atuais de $w_3, w_4$ e $b_3$.
2.  **Cálculo do Erro:** Medimos a distância entre a previsão e o valor real (SSR).
3.  **Backward Pass:** Calculamos os **três gradientes** independentes para descobrir a direção do ajuste para cada parâmetro.
4.  **Atualização:** Modificamos $w_3, w_4$ e $b_3$ simultaneamente usando a taxa de aprendizado.
5. **Verifica a Precisão:** Se os passos de atualização para o $w_3, w_4 eb_3$ forem pequenos demais

O processo se repete até que todos os gradientes sejam próximos de zero (convergência simultânea) ou o maximo de iterações for atingido.

Também armazenamos o histórico dos valores de $w_3, w_4 eb_3$ para a visualização da convergência do algoritmo em gráficos posteriores.

In [11]:
for i in range(max_iterations):
    values_of_b3.append(b3)
    values_of_w3.append(w3)
    values_of_w4.append(w4)

    predictions_for_ssr = neural_network(x_data, b3, w3, w4)
    current_ssr = np.sum((y_data - predictions_for_ssr)**2)
    list_sum_of_squared_residuals.append(current_ssr)

    current_gradient_b3 = calculate_gradient_b3(x_data, b3, w3, w4, y_data)
    list_current_gradient_b3.append(current_gradient_b3)

    current_gradient_w3 = calculate_gradient_w3(x_data, b3, w3, w4, y_data)
    list_current_gradient_w3.append(current_gradient_w3)

    current_gradient_w4 = calculate_gradient_w4(x_data, b3, w3, w4, y_data)
    list_current_gradient_w4.append(current_gradient_w4)

    print(f"""
Step Size b3: {(learning_rate * current_gradient_b3):.4f},
Old b3: {b3:.4f},
New b3: {(b3 - learning_rate * current_gradient_b3):.4f}
Step size w3: {(learning_rate * current_gradient_w3):.4f}
Old w3: {w3:.4f}
New w3: {(w3 - learning_rate * current_gradient_w3):.4f}
Step size w4: {(learning_rate * current_gradient_w4):.4f}
Old w4: {w4:.4f}
New w4: {(w4 - learning_rate * current_gradient_w4):.4f}
""",
              flush=True)

    b3 = b3 - learning_rate * current_gradient_b3
    w3 = w3 - learning_rate * current_gradient_w3
    w4 = w4 - learning_rate * current_gradient_w4

    if np.abs(current_gradient_b3*learning_rate) < precision and \
     np.abs(current_gradient_w3*learning_rate) < precision and \
     np.abs(current_gradient_w4*learning_rate) < precision:
        break

print(f"b3 Final: {b3:.4f}")
print(f"w3 Final: {w3:.4f}")
print(f"w4 Final: {w4:.4f}")
print(f"Gradient b3 Final: {current_gradient_b3:.4f}")
print(f"Gradient w3 Final: {current_gradient_w3:.4f}")
print(f"Gradient w4 Final: {current_gradient_w4:.4f}")
print(f"Total de Iterações: {len(values_of_b3)}")

A saída de streaming foi truncada nas últimas 5000 linhas.
Step Size b3: -0.0012,
Old b3: 2.3845,
New b3: 2.3858
Step size w3: 0.0006
Old w3: -1.0995
New w3: -1.1002
Step size w4: 0.0012
Old w4: -2.0823
New w4: -2.0836


Step Size b3: -0.0012,
Old b3: 2.3858,
New b3: 2.3870
Step size w3: 0.0006
Old w3: -1.1002
New w3: -1.1008
Step size w4: 0.0012
Old w4: -2.0836
New w4: -2.0848


Step Size b3: -0.0012,
Old b3: 2.3870,
New b3: 2.3882
Step size w3: 0.0006
Old w3: -1.1008
New w3: -1.1014
Step size w4: 0.0012
Old w4: -2.0848
New w4: -2.0860


Step Size b3: -0.0012,
Old b3: 2.3882,
New b3: 2.3894
Step size w3: 0.0006
Old w3: -1.1014
New w3: -1.1021
Step size w4: 0.0012
Old w4: -2.0860
New w4: -2.0872


Step Size b3: -0.0012,
Old b3: 2.3894,
New b3: 2.3906
Step size w3: 0.0006
Old w3: -1.1021
New w3: -1.1027
Step size w4: 0.0012
Old w4: -2.0872
New w4: -2.0884


Step Size b3: -0.0012,
Old b3: 2.3906,
New b3: 2.3918
Step size w3: 0.0006
Old w3: -1.1027
New w3: -1.1033
Step size w4: 0.0012
Old

### Animação da Convergência (Ajuste de $w_3, w_4$ e $b_3$)

O código a seguir gera a visualização do treinamento da **Rede Neural** otimizando a camada de saída completa ($w_3, w_4$ e $b_3$).

A função de atualização (`update`) reconstrói a curva de previsão aplicando os novos pesos e viés às saídas fixas das camadas ocultas:

$$
\hat{y} = (h_1 \cdot w_{3(\text{atual})} + h_2 \cdot w_{4(\text{atual})}) + b_{3(\text{atual})}
$$

Diferente do exemplo anterior, tanto o $b_3$ quanto $w_3, w_4$ são alterados continuamente.

In [12]:
fig, ax = plt.subplots(figsize=(6, 6))
ax.scatter(x_data, y_data, color='blue', zorder=5, label='Pontos Originais')
ax.set_title('Gradiente Descendente: Convergência do Parâmetro b3')
ax.set_xlabel('height')
ax.set_ylabel('Eixo Y')
ax.legend()
ax.grid(True)

x_curve_anim = np.linspace(min(x_data) - 0.2, max(x_data) + 0.2, 100)

curve, = ax.plot(x_curve_anim, neural_network(x_curve_anim, values_of_b3[0], values_of_w3[0], values_of_w4[0]), color='red', label='Curva da Rede Neural')
iteration_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, color='green')

def update(frame):
    current_b3 = values_of_b3[frame]
    current_w3 = values_of_w3[frame]
    current_w4 = values_of_w4[frame]
    y_curve_anim = neural_network(x_curve_anim, current_b3, current_w3, current_w4)
    curve.set_ydata(y_curve_anim)
    iteration_text.set_text(f'Iteração: {frame+1}\nb3: {current_b3:.4f}\nw3: {current_w3:.4f}\nw4: {current_w4:.4f}')
    return curve, iteration_text,

ani = FuncAnimation(fig, update, frames=len(values_of_b3), interval=8, blit=True)

plt.close(fig)
HTML(ani.to_jshtml())

Output hidden; open in https://colab.research.google.com to view.